<a href="https://colab.research.google.com/github/Macia-Comella/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Maci%C3%A0_Comella_Barb%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Macià Comella Barbé  <br>
Url: https://github.com/Macia-Comella/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Maci%C3%A0_Comella_Barb%C3%A9.ipynb <br>
Google Colab: https://colab.research.google.com/drive/1FGZOnWffgCmwZOaOAiDOFlKo6aX0BwOK?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:

Número de actores: 10  
Número de tomas : 30  
Actores/Tomas : https://bit.ly/36D8IuK

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

### Respuesta

**Espacio de soluciones:**  
Representamos el espacio de soluciones con una lista de listas que contiene las sesiones programadas cada día. Cada lista representa un día y éste contiene las sesiones de dicho día. El tamaño de las listas deberá limitarse a 6 ya que no se pueden programar más de 6 tomas por día.

Asociada a esta lista de programación de tomas generamos una lista de listas con los actores que deben desplazarse al estudio cada día siguiendo la misma lógica (cada lista representa un día y contiene los actores necesarios para tal día)

Trabajamos con soluciones compactas intentando encabir el máximo de tomas en un día. Como el límite es 6 tomas/día y hay 30 tomas a programar se pueden compactar en 5 días. Se toma este enfoque ya que, en general, repartir las tomas a lo largo de más días incrementa la probabilidad de que actores deban desplazarse más días al estudio. Visto desde otro punto de vista, una vez hemos programado una toma en un día, los actores necesarios para esa toma ya están desplazados al estudio, por lo que todas las tomas adicionales que puedan grabar no añaden coste adicional.

In [1]:
import random
import math
from copy import deepcopy

In [2]:
# Carga datos actores
planificacion = {'1': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                 '2': [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                 '3': [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
                 '4': [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
                 '5': [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
                 '6': [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                 '7': [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                 '8': [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
                 '9': [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
                 '10': [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
                 '11': [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
                 '12': [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
                 '13': [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                 '14': [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                 '15': [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
                 '16': [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
                 '17': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 '18': [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                 '19': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 '20': [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                 '21': [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                 '22': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                 '23': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 '24': [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                 '25': [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                 '26': [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
                 '27': [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                 '28': [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                 '29': [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
                 '30': [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]}

# Transformación del formato de los actores para tener el identificador del actor en lugar de una tabla binaria
for k in planificacion:
  lista = []
  for numero, actor in enumerate(planificacion[k]):
    if actor == 1:
      lista.append(numero+1)
  planificacion[k] = lista

print(planificacion)

{'1': [1, 2, 3, 4, 5], '2': [3, 4, 5], '3': [2, 5, 7], '4': [1, 2, 7, 8], '5': [2, 4, 8], '6': [1, 2, 4, 5], '7': [1, 2, 4, 5], '8': [1, 2, 6], '9': [1, 2, 4], '10': [1, 2, 6, 9], '11': [1, 2, 3, 5, 8], '12': [1, 2, 3, 4, 6], '13': [1, 4, 5], '14': [1, 3, 6], '15': [1, 2, 7], '16': [4, 10], '17': [1, 3], '18': [3, 6], '19': [1, 3], '20': [1, 3, 4, 5], '21': [6, 8], '22': [1, 2, 3, 4], '23': [1, 3], '24': [3, 6], '25': [1, 2, 4, 10], '26': [1, 3, 5, 9], '27': [4, 5], '28': [1, 4], '29': [1, 5, 6], '30': [1, 4]}


In [3]:
# Espacio de soluciones

def genera_programacion_aleatoria():
  programacion = [[]]
  tomas_a_programar = list(range(1,31))

  while len(tomas_a_programar) > 0:
    for dia in programacion:
      programado = False
      if len(dia) < 6:
        toma_programada = random.choice(tomas_a_programar)
        dia.append(toma_programada)
        tomas_a_programar.remove(toma_programada)
        programado = True

    if not programado:
      programacion.append([])

  return programacion

In [4]:
def programa_actores(programacion, plan):
  programacion_actores = []
  for dia, tomas_dia in enumerate(programacion):
    programacion_actores.append([])
    for toma in tomas_dia:
      for tomas_plan in plan[str(toma)]:
        programacion_actores[dia].append(tomas_plan)

  for dia in range(len(programacion_actores)):
    programacion_actores[dia] = set(programacion_actores[dia])

  return programacion_actores


In [5]:
programacion = genera_programacion_aleatoria()

prog_actores = programa_actores(programacion, planificacion)

dia = 1
for tomas, actores in zip(programacion, prog_actores):
  print(f"El día {dia} se programan las tomas {tomas}, por lo tanto se necesitan los actores {actores}")
  dia += 1

El día 1 se programan las tomas [10, 11, 16, 14, 20, 6], por lo tanto se necesitan los actores {1, 2, 3, 4, 5, 6, 8, 9, 10}
El día 2 se programan las tomas [1, 21, 5, 26, 8, 7], por lo tanto se necesitan los actores {1, 2, 3, 4, 5, 6, 8, 9}
El día 3 se programan las tomas [27, 23, 15, 12, 9, 13], por lo tanto se necesitan los actores {1, 2, 3, 4, 5, 6, 7}
El día 4 se programan las tomas [2, 4, 22, 24, 25, 18], por lo tanto se necesitan los actores {1, 2, 3, 4, 5, 6, 7, 8, 10}
El día 5 se programan las tomas [19, 28, 30, 3, 29, 17], por lo tanto se necesitan los actores {1, 2, 3, 4, 5, 6, 7}


**Función objetivo:** la función objetivo a minimizar es el coste del traslado de los actores al estudio de grabación. Definimos el coste de convocar a un actor al estudio de gravación con valor 1 por cada día que un actor deba desplazarse. Con esta definición el cálculo del coste a minimizar es el número de actores convocados cada día.  

A continuación definimos la función coste y calculamos el coste de la solución aleatoria generada en el apartado anterior

In [ ]:
print(prog_actores)

[{1, 2, 3, 4, 5, 6, 8, 10}, {1, 2, 3, 4, 5, 6, 8}, {1, 2, 3, 4, 5, 6, 7, 8, 10}, {1, 2, 3, 4, 5, 6, 9}, {1, 2, 3, 4, 5, 6, 7}]


In [6]:
def coste(prog_actores):
  coste = 0
  for dia in prog_actores:
    coste += len(dia)

  return coste

In [7]:
coste_prog_aleatoria = coste(prog_actores)
print(f"El coste de la programación aleatoria es de {coste_prog_aleatoria} unidades")

El coste de la programación aleatoria es de 40 unidades


**Restricciones:** de la forma como hemos modelizado el espacio de soluciones el enunciado del problema nos impone dos restricciones:
1. No pueden programarse más de 6 tomas cada día y
2. Deben programarse las 30 tomas.

La primera restricción la implementamos en todas las funciones que generen soluciones o modificaciones de las soluciones, la longitud de la lista de soluciones programadas por día no puede ser superior a 6.  

La segunda restricción la implementamos asegurando que las soluciones incluyen el total de las 30 tomas a programar.

Adionalmente, el enunciado nos proporciona una tabla que condiciona los actores que deben desplazarse al estudio cada día en función de las tomas que están programadas ese día. Esta condición en realidad no impone ninguna restricción, sinó que influirá en el valor de la función objetivo.

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

### Respuesta
El espacio de soluciones completo incluye todas las posibles combinaciones de tomas programadas por día, teniendo en cuenta que se programan 6 tomas por día para un total de 5 días.  

Para el cómputo del espacio de soluciones consideramos que si las tomas se programan en diferente orden dentro de un mismo día se consideran soluciones diferentes, aunque tengan el mismo valor de coste. Lo mismo ocurre si intercambiamos el orden de días. Siguiendo esta lógica el espacio de soluciones se contabiliza del siguiente modo:

1. El 1r día hay 30 tomas a programar, de las cuales se programan 6. No se puede repetir pero el orden de las tomas importa, por lo tanto estamos ante un caso de variación sin repetición con fórmula: 30!/(30-6)! = 30!/24! = 30 * 29 * 28 * 27 * 26 * 25
2. El 2o día quedan 24 tomas a programar. Aplicamos la misma función actualizada para las tomas restantes: 24!/(24-6)! = 24!/18! = 24 * 23 * 22 * 21 * 20 * 19

Siguiendo este proceso hasta programar las 30 tomas el cómputo total de soluciones es:  
30!/24! * 24!/18! * 18!/12! * 12!/6! * 6! = 30!

Generalizando la solución, con las mismas restricciones, el orden de complejidad es factorial, concretamente O(n) = n!

In [ ]:
# A continuación se calcula el valor concreto de soluciones para n = 30 -> factorial de 30
math.factorial(30)

265252859812191058636308480000000

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

### Respuesta

En una primera aproximación utilizaremos una combinación de búsqueda local y multiarranque para mejorar una solución aleatoria inicial. El multi-arranque nos aporta diversificación al empezar en varios puntos diferentes, mientras que la búsqueda local nos aporta intensificación en estos puntos iniciales aleatorios. Como la búsqueda local encuentra soluciones bastante rápido  podemos hacer bastantes arranques para empezar en sitios diferente.  

En una revisión del código, añadimos una componente de la búsqueda tabú: como la búsqueda local siempre encuentra la misma solución dada una solución inicial, restringimos las soluciones aleatorias iniciales a que no se repitan con las soluciones generadas anteriormente. De esta forma aseguramos que la generación aleatoria no nos lleva a invertir recursos en caminos ya explorados

In [61]:
# Propuesta 1: combinar multi-arranque con búsqueda local
def genera_vecinos_2opt(programacion, plan):
  mejor_solucion = programacion
  prog_actores = programa_actores(programacion, plan)
  mejor_coste = coste(prog_actores) # Usamos la programación actual como valor a comparar. Si la generada no es mejor que la actual no queremos cambiar

  vecina = deepcopy(programacion)
  for dia in range(len(programacion)-1):
    for toma in range(len(programacion[dia])):
      for toma_dia_cambio in range(len(programacion[dia])):

        vecina[dia][toma] = programacion[dia+1][toma_dia_cambio]
        vecina[dia+1][toma_dia_cambio] = programacion[dia][toma]

        prog_actores_vec = programa_actores(vecina, plan)
        coste_vec = coste(prog_actores_vec)

        if coste_vec < mejor_coste:
          mejor_solucion = vecina
          mejor_coste = coste_vec

        vecina = deepcopy(programacion)

  return mejor_solucion

def multiarranque_busq_local(plan, intentos):
  mejor_sol_global = []
  mejor_coste_global = float('inf')
  sol_ya_intentadas = []

  for _ in range(intentos):
    parada = False

    sol_propuesta = genera_programacion_aleatoria()

    while sol_propuesta in sol_ya_intentadas:
      sol_propuesta = genera_programacion_aleatoria()

    mejor_sol_local = sol_propuesta
    mejor_coste_local = float('inf')
    sol_ya_intentadas.append(sol_propuesta)

    while not parada:
      sol_busq_local = genera_vecinos_2opt(mejor_sol_local, plan)
      prog_actores_busq_local = programa_actores(sol_busq_local, plan)
      coste_sol_busq_local = coste(prog_actores_busq_local)

      if coste_sol_busq_local < mejor_coste_local:
        mejor_sol_local = sol_busq_local
        mejor_coste_local = coste_sol_busq_local

      else:
        parada = True

    if mejor_coste_local < mejor_coste_global:
      mejor_sol_global = mejor_sol_local
      mejor_coste_global = mejor_coste_local

  return mejor_sol_global

In [56]:
print(programacion)
# print(coste_prog_aleatoria)

sol_busq_local = genera_vecinos_2opt(programacion, planificacion)
print(sol_busq_local)

[[10, 11, 16, 14, 20, 6], [1, 21, 5, 26, 8, 7], [27, 23, 15, 12, 9, 13], [2, 4, 22, 24, 25, 18], [19, 28, 30, 3, 29, 17]]
[[10, 26, 16, 14, 20, 6], [1, 21, 5, 11, 8, 7], [27, 23, 15, 12, 9, 13], [2, 4, 22, 24, 25, 18], [19, 28, 30, 3, 29, 17]]


In [64]:
print(f"Programación aleatoria: {programacion}")
print(f"Coste programación aleatoria: {coste_prog_aleatoria}")

sol_multi_busq_local = multiarranque_busq_local(planificacion, intentos=500)
print(sol_multi_busq_local)

prog_actores_multi_busq_local = programa_actores(sol_multi_busq_local, planificacion)
coste_multi_busq_local = coste(prog_actores_multi_busq_local)

print(coste_multi_busq_local)

# Habiendo lanzado varias ejecuciones, con 500 intentos vemos que la mejor solución a
# la que podemos llegar con esta heurística está en el entorno de 29 unidades de coste

Programación aleatoria: [[10, 11, 16, 14, 20, 6], [1, 21, 5, 26, 8, 7], [27, 23, 15, 12, 9, 13], [2, 4, 22, 24, 25, 18], [19, 28, 30, 3, 29, 17]]
Coste programación aleatoria: 40
[[26, 10, 1, 29, 22, 12], [27, 28, 7, 9, 30, 16], [24, 23, 14, 19, 18, 17], [15, 5, 3, 6, 4, 25], [20, 8, 13, 21, 2, 11]]
29


In [66]:
# Como sabemos que no podemos repetir caminos ya explorados y el tiempo de ejecucion
# con el hardware de Colab no es grande lanzamos una ejecución con 2000 intentos para
# obtener una solución bastante buena con esta heurística

sol_multi_busq_local = multiarranque_busq_local(planificacion, intentos=2000)
print(sol_multi_busq_local)

prog_actores_multi_busq_local = programa_actores(sol_multi_busq_local, planificacion)
coste_multi_busq_local = coste(prog_actores_multi_busq_local)

print(coste_multi_busq_local)

# El tiempo de ejecución ha sido de 1 minuto

[[10, 6, 8, 12, 9, 26], [1, 7, 16, 20, 2, 25], [23, 24, 18, 19, 14, 17], [27, 29, 28, 13, 30, 21], [5, 22, 15, 11, 3, 4]]
28
